In [62]:
import pandas as pd
import numpy as np
import dart_fss as dart

In [63]:
#사용자에게 정보 제공하기 위한 데이터프레임 만들기
col_list = ['매출액', '영업이익', '당기순이익', '영업이익률', '순이익률', 'EPS', 
            'ROE', 'ROA', '자산총계', '부채총계', '자본총계', '부채비율']
index_list = [company + '2022', company + '2021', company + '2020']
df = pd.DataFrame(columns = col_list, index = index_list)

In [70]:
#사용자로부터 원하는 기업명 입력받기
company = input()

삼성전자


In [71]:
#OPEN DART API key 설정
api_key = 'c60afaa1f54d82997cd25ff8c6712e5421ad72f5'
dart.set_api_key(api_key = api_key)

#DART에 공시된 회사 리스트 불러오기
corp_list = dart.get_corp_list()

#삼성전자 검색
samsung = corp_list.find_by_corp_name(company, exactly=True)[0]

#2022년 연간 연결재무제표 불러오기
fs = samsung.extract_fs(bgn_de='20230101')

#손익계산서 불러오기
df_is = fs['is']

#index와 data 분리하여 정리
index = df_is['[D310000] Income statement, by function of expense - Consolidated financial statements (Unit: KRW)']['label_ko']
income_statement = df_is[['20220101-20221231','20210101-20211231','20200101-20201231']].droplevel(1, axis = 1)
income_statement.columns = [company+'2022', company+'2021', company+'2020']
income_statement.index = index

#재무상태표 불러오기
df_bs = fs['bs']
index = df_bs['[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW)']['label_ko']
balance_sheet = df_bs[['20221231','20211231','20201231']].droplevel(1, axis = 1)
balance_sheet.columns = [company+'2022', company+'2021', company+'2020']
balance_sheet.index = index

Annual reports:   0%|          | 0/1 [00:00<?, ?report/s]

In [74]:
#손익 계산서와 재무상태표에서 필요한 정보 가져오기
for i in index_list:
    #매출액
    df.loc[i, '매출액'] = income_statement.loc['수익(매출액)', i]
    #영업이익
    df.loc[i, '영업이익'] = income_statement.loc['영업이익', i]
    #당기순이익
    df.loc[i, '당기순이익'] = income_statement.loc['당기순이익(손실)', i]
    #영업이익률
    df.loc[i, '영업이익률'] = (income_statement.loc['영업이익', i] / income_statement.loc['수익(매출액)', i])
    #순이익률
    df.loc[i, '순이익률'] = (income_statement.loc['당기순이익(손실)', i] / income_statement.loc['수익(매출액)', i])
    #EPS
    df.loc[i, 'EPS'] = income_statement.loc['기본주당이익(손실)', i]
    #ROE
    df.loc[i, 'ROE'] = (income_statement.loc['당기순이익(손실)', i] / balance_sheet.loc['자본총계', i])
    #ROA
    df.loc[i, 'ROA'] = (income_statement.loc['당기순이익(손실)', i] / balance_sheet.loc['자산총계', i])
    #자산총계
    df.loc[i, '자산총계'] = balance_sheet.loc['자산총계', i]
    #부채총계
    df.loc[i, '부채총계'] = balance_sheet.loc['부채총계', i]
    #자본총계
    df.loc[i, '자본총계'] = balance_sheet.loc['자본총계', i]
    #부채비율
    df.loc[i, '부채비율'] = (balance_sheet.loc['부채총계', i] / balance_sheet.loc['자본총계', i])

In [75]:
df

,매출액,영업이익,당기순이익,영업이익률,순이익률,EPS,ROE,ROA,자산총계,부채총계,자본총계,부채비율
삼성전자2022,"302,231,360,000,000.0","43,376,630,000,000.0","55,654,077,000,000.0",0.1435212745626397,0.1841439518387503,"8,057.0",0.15688270366610474,0.1241102485061103,"448,424,507,000,000.0","93,674,903,000,000.0","354,749,604,000,000.0",0.26405921794911996
삼성전자2021,"279,604,799,000,000.0","51,633,856,000,000.0","39,907,450,000,000.0",0.18466727389754137,0.14272805811176367,"5,777.0",0.1308870417553489,0.09354306332833122,"426,621,158,000,000.0","121,721,227,000,000.0","304,899,931,000,000.0",0.39921697128885214
삼성전자2020,"236,806,988,000,000.0","35,993,876,000,000.0","26,407,832,000,000.0",0.15199668009797077,0.1115162699506148,"3,841.0",0.0956985753432632,0.0698184511490266,"378,235,718,000,000.0","102,287,702,000,000.0","275,948,016,000,000.0",0.3706774322305691
